# Innovationen durch Open Data 
## Den Wert von offenen Daten aufdecken und erschließen

Im Vortrag haben wir Beispiele von offenen Daten gezeigt bekommen, welche von
- Städten und Kommunen,
- zivilgesellschaftlichen Projekten oder
- wissenschaftlichen Communities
veröffentlicht werden.

Wir möchten nun hands-on Beispiele für jeden dieser Datenurheber vorstellen, damit ihr einen Eindruck von der Qualität, möglichen Anwendungen aber auch den Herausforderungen in der Verwendung dieser Daten erhaltet.

Unsere Agenda sieht wie folgt aus:
1. Visualisierung von Datensätzen des Open Data-Portals der Stadt Bielefeld
2. Zugriff auf Daten der Feinstaubsensoren in Bielefeld inkl. einer interaktiven Karte
3. Linked Data in Wikidata

## Pakete laden
- `pandas`: Verarbeitung, Verwaltung und Analyse von Daten
- `plotly`: Erstellung interaktiver Grafiken; eine Vielzahl von Beispielen wie verschiedene Datentypen zur Anzeige gebracht werden können, sind dort auf der [offiziellen Webseite](https://plotly.com/python/) zu finden
- `requests`: Führe HTTP-Befehle aus um z.B. Webseiten abzufragen
- `datetime`: Standardisierte Formate und Funktionen für Zeitstempel
- `json`: Verarbeitung von JSON-Dateien
- `pywikibot`: Vielfältige Funktionen zur Abfrage von Wikipedia und Wikidata

In [ ]:
import pandas as pd
import plotly.express as px
import requests
from datetime import datetime, timedelta
import json
import sys
import pywikibot

## Open-Data-Portal Bielefeld

Zunächst möchten wir uns Daten des Open-Data-Portals der Stadt Bielefeld zur Anzeige bringen. Hierzu verwenden wir zunächst die Wahlergebnisse der Bundestagswahl 2017.

### Wahlergebnisse der Bundestagswahl 2017
Auf [dieser Webseite](https://open-data.bielefeld.de/dataset/wahlen-und-abstimmungen-bielefeld) finden sich alle Datensätze zu Wahlen und Abstimmungen der Stadt. Ganz unten auf der Seite finden sich die aktuellsten Daten, und dort sehen wir 

<img src="./img/Daten_Download_1.png" width="750" height="250" align="left">

Fahren wir nun mit dem Mauszeiger über den Button `Download`, so sehen wir die eigentliche URL des Datensatzes:

<img src="./img/Daten_Download_2.png" width="900" height="250" align="left">

Diese URL fügen wir in die Funktion zum Einlesen einer CSV-Datei ein.

In [ ]:
url_bundestagswahl2017 = 'https://open-data.bielefeld.de/sites/default/files/BTW 2017_1.csv'

try:
    df_btw = pd.read_csv(url_bundestagswahl2017)
except:
    print("Oops! Dies ist eine ungültige URL...\nFehlerklasse:", sys.exc_info()[0])


Leider erhalten wir die Fehlermeldung, dass kein Leerzeichen in einer URL als Eingabe dieser Funktion vorhanden sein darf. 

Ersetzen wir also das Leerzeichen mit `%20`, welches wir aus reinem Zufall kennen.

In [ ]:
url_bundestagswahl2017 = 'https://open-data.bielefeld.de/sites/default/files/BTW%202017_1.csv'

try:
    df_btw = pd.read_csv(filepath_or_buffer=url_bundestagswahl2017)
except:
    print("Oops! Die Zeichen in der Datei können nicht interpretiert werden...\nFehlerklasse:", sys.exc_info()[0])

Ein andere Fehler taucht auf. Es wird eine Encodierung angesprochen und wir haben den Verdacht, dass die CSV-Datei mithilfe von Word auf einer Windows-Maschine erstellt sein könnte. Wenn nicht explizit darauf geachtet wird, werden so leider Dateien erstellt, welche nicht ein Standard-Format besitzen.

Aber wieder wissen wir uns zu helfen und tippen darauf, dass wir die Encodierung `ANSI` verwenden sollten. Da ANSI als Encoding Format nicht zu den Standard Python Encodings gehört, nutzen wir "palmos", welches für ANSI Encoding in Python verwendet werden kann.

In [ ]:
url_bundestagswahl2017 = 'https://open-data.bielefeld.de/sites/default/files/BTW%202017_1.csv'
try:
    df_btw = pd.read_csv(filepath_or_buffer=url_bundestagswahl2017, encoding='palmos')
except:
    print("Oops! Das Pandas-Paket kann die Datei nicht verarbeiten...\nFehlerklasse:", sys.exc_info()[0])

Und noch ein Fehler...

Irgendetwas scheint mit der erwarteten Anzahl der Spalten nicht zu stimmen. Obwohl das Datenformat CSV (comma-separated values), wird häufig ein Semikolon anstelle eines Kommas eingesetzt. Probieren wir es also aus.

In [ ]:
url_bundestagswahl2017 = 'https://open-data.bielefeld.de/sites/default/files/BTW%202017_1.csv'
df_btw = pd.read_csv(filepath_or_buffer=url_bundestagswahl2017, sep=';', encoding='palmos')
df_btw.head()

Um einen besseren Eindruck über die verfügbaren Daten zu erhalten, lassen wir uns alle Spaltennamen anzeigen:

In [ ]:
print(list(df_btw.keys()))

Wenn wir uns lediglich für die prozentuale Verteilung der Stimmen interessieren, so können wir die entsprechenden Spalten filtern. Leider ist im Portal nicht beschrieben, inwiefern sich die Spalten mit dem Präfix `Z_` von denjenigen ohne unterscheiden. Wir wählen diejenigen ohne Präfix.

In [ ]:
list_btw_prc = [x for x in list(df_btw.keys()) if '_Proz' in x and 'Z_' not in x]
print(list_btw_prc)

Wir erzeugen einen Dataframe mit nur obigen Spalten und der Wahlbezirksnummer.

In [ ]:
df_btw_prc = df_btw[['Nr'] + list_btw_prc].copy()
df_btw_prc.head()

Leider ist in dem Datensatz nicht ein international gültiges Nummernformat verwendet worden, sodass wir als Text erkannte Zahlen wie `2,7` in echte Zahlen wie `2.7` umwandeln.

In [ ]:
for col in list_btw_prc:
    if df_btw_prc[col].dtype == object:
        df_btw_prc[col] = df_btw_prc[col].str.replace(',', '.').astype(float)

Nach der Korrektur erhalten wir folgenden Dataframe:

In [ ]:
df_btw_prc.head()

Um die Verteilung von Wahlbezirk 1.1 darzustellen, legen wir einen eigenen Dataframe an.

In [ ]:
df_btw_1_1 = df_btw_prc[list_btw_prc].iloc[0].to_frame()
df_btw_1_1.reset_index(inplace=True)
df_btw_1_1 = df_btw_1_1.rename(columns = {'index':'Partei', 0:'Proz'})
df_btw_1_1['Partei'] = df_btw_1_1['Partei'].str.replace('_Proz', '')

Ein Kreisdiagramm mit der prozentualen Stimmenverteilung in Wahlbezirk 1.1 kann nach dieser Vorarbeit einfach dargestellt werden:

In [ ]:
fig = px.pie(df_btw_1_1.loc[df_btw_1_1['Proz'] > 0.02], values='Proz', names='Partei', title='Wahlbezirk 1.1')
fig.show()

Es ließen sich noch eine Vielzahl an weiteren Grafiken und Statistiken bilden, allerdings warten noch andere Datensätze auf uns.

Take-away-message:
- die maschinelle Zugriffsmöglichkeit auf offene Daten sollte standardisiert und gut beschrieben sein
- Metadaten - also die Beschreibung der Inhalte einer Datei - sind zur Interpretation notwendig
- standardisierte Formate sollten verwendet werden um längliche Datenprozessierung unnötig zu machen

### Demographische Daten

Nun betrachten wir die demographischen Daten der Stadt Bielefeld und verwenden hierfür die Daten [dieser Webseite](https://open-data.bielefeld.de/dataset/demographie-und-statistische-gebietsgliederung) . 

Die entsprechende URL erhalten wir per Mouse-Over über den Download-Button.

<img src="./img/Daten_Download_3.png" width="750" height="450" align="left">

Zunächst laden wir die Daten ein und vermuten, dass wir die obigen Parameter zum fehlerfreien Einlesen auch hier verwenden können.

In [ ]:
url_alter = 'https://open-data.bielefeld.de/sites/default/files/altersstruktur_stadtbezirke_31122019_0.csv'
df_alter = pd.read_csv(filepath_or_buffer=url_alter, sep=';', encoding='palmos')
df_alter.head()

Betrachten wir zunächst wieder alle Spaltennamen:

In [ ]:
list_alter_spaltennamen = list(df_alter.keys())
print(list_alter_spaltennamen)

**STOP** An dieser Stelle fällt uns auf, dass sich die Daten im Vergleich zum Open-Data-Day 2021 (Anfang März) geändert haben. Wo finden wir Informationen hierzu, vielleicht gibt es je eine Datei-Versionierung?

Das Portal bietet eine JSON-API an, betrachten wir also die Informationen zu diesem Datensatz hierin:

<img src="./img/Daten_Download_4.png" width="800" height="350" align="left">

Leider können wir keine Änderungsinformationen ablesen. Vermutlich ändern sich Felder wie `last_modified` lediglich, wenn der Eintrag nicht aber der hinterlegte Datensatz aktualisiert wird.

Schauen wir uns nochmals im Portal um, so finden wir den gewünschten Datensatz in einem anderen Eintrag, nämlich unter [Statistische Kurzinformation zur Bevölkerung 2013 bis 2021](https://open-data.bielefeld.de/dataset/statistische-kurzinformation-zur-bev%C3%B6lkerung-2013-bis-2021).

In [ ]:
url_alter = 'https://open-data.bielefeld.de/sites/default/files/altersstruktur_halbjaehrlich2013bisMitte2021.csv'
df_alter = pd.read_csv(filepath_or_buffer=url_alter, sep=';', encoding='palmos')
df_alter.head()

Da so lange Spaltennamen sehr unhandlich sind, weisen wir den Daten einfach neue zu.

In [ ]:
list_alter_neueSpaltennamen = ['jahr', 'stichtag', 'gebiet', 'gesamt', '0_1', '1_3', '3_5', '5_6', '6_10', '10_14', '14_15', '15_18', '18_21', '21_25', '25_30', '30_40', '40_45', '45_60', '60_65', '65_70', '70_75', '75_', 'durchschnittsalter']
df_alter.columns = list_alter_neueSpaltennamen
df_alter.head()

Um Daten im Anschluss korrekt anzeigen zu lassen, müssen wir die Datumsangabe von `stichtag` in ein Datenformat übertragen:

In [ ]:
df_alter['stichtag'] = pd.to_datetime(df_alter['stichtag'], format='%Y%m%d')

Nun sind wir in der Position für den Stadtbezirk Mitte die Demographie der letzten Jahre anzuzeigen:

In [ ]:
fig__alter_mitte = px.bar(df_alter[df_alter['gebiet']=='Stadtbezirk Mitte'], x='stichtag', y='gesamt')
fig__alter_mitte.show()

Interessant ist natürlich auch die Entwicklung in der gesamten Stadt, also binden wir alle Stadtbezirke ein:

In [ ]:
fig__alter = px.bar(df_alter, x='stichtag', y='gesamt', color='gebiet')
fig__alter.show()

Oh, anscheinend gibt es einen Stadtbezirk, welchen es per Definition gar nicht gibt, nämlich `Bielefeld gesamt`. Das überprüfen wir, indem wir uns alle eindeutigen Werte der Spalte `gebiet` anzeigen lassen:

In [ ]:
print(df_alter['gebiet'].unique())

Für eine korrekte Darstellung müssen wir also `Bielefeld gesamt` ausschließen und erhalten:

In [ ]:
df_alter = df_alter[df_alter['gebiet']!='Bielefeld insgesamt']
fig__alter_2 = px.bar(df_alter, x='stichtag', y='gesamt', color='gebiet')
fig__alter_2.show()

Take-away-message:
- die URLs von Daten sollten statisch sein
- Versionierung von Datensätzen schafft zusätzliche Transparenz
- es benötigt Zeit sich in Datenportale einzuarbeiten

## Zugriff auf Daten der Feinstaubsensoren in Bielefeld

Im Rahmen von [Code for Bielefeld](https://codefor.de/bielefeld) beschäftigen wir uns unter anderem mit der Erhebung und Auswertung von Feinstaubdaten im Stadtgebiet. Wer sich einen Eindruck über die aktuell im Einsatz befindlichen Sensoren machen möchte, der kann diese auf [sensor.community](https://sensor.community/de/) nachschauen.

Wer selbst Interesse hat einen Feinstaubsensor in Betrieb zu nehmen, der findet [hier](https://sensor.community/de/sensors/airrohr/) eine Anleitung oder kann sich den [NW-Beitrag](https://www.nw.de/lokal/bielefeld/mitte/22873870_Sich-einfach-mal-selbst-einen-Feinstaubsensor-bauen-und-auch-aufhaengen.html) zu unserem Familienworkshop Anfang Oktober letzten Jahres durchlesen.

Die Sensoren übermitteln per WLAN die Messdaten alle 5 Minuten an die Plattform von [sensor.community](https://sensor.community/de/), welche wir im Folgenden nutzen.

### Datenabfrage für einzelne Sensoren

Gehen wir also auf [sensor.community](https://sensor.community/de/) und wählen einen beliebigen Sensor aus.

<img src="./img/Sensor_2.png" width="800" height="600" align="left">

Klicken wir auf den Sensor drauf, können wir uns die gemessenen Daten der letzten 24 Stunden anzeigen lassen.

<img src="./img/Sensor_1.png" width="800" height="600" align="left">

Entsprechend versuchen wir nun diese Daten selbst abzufragen. Zum einen haben wir die Möglichkeit, die Daten der letzten 5 Minuten eines beliebigen Sensors per Rest-API herunterzuladen.

In [ ]:
url = "https://data.sensor.community/airrohr/v1/sensor/49366/"
res = requests.get(url)
print(res.text)

Doch leider sind 5 Minuten kein allzu langer Zeitraum. Eine zweite Möglichkeit besteht darin, bereits archivierte Daten von dem entsprechenden FTP-Server abzufragen.

In [ ]:
df_sensor_49366 = pd.read_csv('https://archive.sensor.community/2021-08-27/2021-08-27_sds011_sensor_49366.csv', sep=';')
df_sensor_49366.head()

Wie zuvor müssen wir für die zukünftige Nutzung den Zeitstempel konvertieren.

In [ ]:
df_sensor_49366['timestamp'] = pd.to_datetime(df_sensor_49366['timestamp'], format='%Y-%m-%dT%H:%M:%S')

Als ersten Plot stellen wir die Daten der Spalte `P1` zum Zeitstempel dar:

In [ ]:
fig = px.line(df_sensor_49366, x='timestamp', y="P1")
fig.show()

An dieser Stelle sei erwähnt, dass die Spalte `P1` für die Feinstaubdaten des Typs `PM10` stehen, und `P2` für die des Typs `PM2.5`. Dementsprechend handelt es sich bei `P2` um die gesundheitlich schädlicheren Daten, da diese Partikel besonders fein sind und in die Lunge eindringen können.

Bilden wir also `P1` und `P2` gemeinsam ab:

In [ ]:
fig = px.line(
    df_sensor_49366[['timestamp','P1','P2']], 
    x="timestamp",
    y=df_sensor_49366[['timestamp','P1','P2']].columns,
    title='Feinstaubdaten des Sensors 49366 am 27.08.2021')
fig.show()

Was müssen wir tun, um alle Daten der letzten 30 Tage abzurufen? Nun, wir müssen wir jeden Tag die entsprechende URL generieren, die einzelnen Datensätze herunterladen und zuletzt zusammensetzen. Gehen wir es also an.

Zunächst generieren wir die Datumsangaben der letzten 30 Tage und formatieren diese entsprechend der Ziel-URL.

In [ ]:
from datetime import datetime
end = datetime.today()
start = end - timedelta(days=30)
list_dates = pd.date_range(start, end).tolist()
list_dates = [x.strftime("%Y-%m-%d") for x in list_dates]
print(list_dates[:5])

Zusätzlich benötigen wir den Namen des Sensortypens, um die Daten aus dem Archiv laden zu können.

In [ ]:
# Sensortyp des Sensors 49366 für Download URL holen
df_sensor_type_49366 = df_sensor_49366['sensor_type']
sensor_type_49366 = df_sensor_type_49366[0].lower()
print(sensor_type_49366)

Dann erzeugen wir eine Liste mit allen URLs unter zu Hilfe nahme der obigen Datumsliste:

In [ ]:
url_front = 'https://archive.sensor.community/'
url_back = '_sensor_49366.csv'
type_sensor = sensor_type_49366
list_urls_sensor_49366 = [url_front + x + '/' + x + '_' + type_sensor + url_back for x in list_dates]
list_urls_sensor_49366[:5]

Da Daten erst mit einem Tag Versatz archiviert werden, müssen wir die letzte URL ausschließen, welche auf den heutigen Tag zugreifen würde:

In [ ]:
list_urls_sensor_49366.pop()

Nun können wir mithilfe einer `for`-Schleife alle Daten herunterladen und in einem Dataframe zusammenfassen.

**Achtung**: Dieser Schritt kann einige Zeit in Anspruch nehmen.

In [ ]:
list_df_sensor_49366 = list()
for url in list_urls_sensor_49366:
    try:
        list_df_sensor_49366.append(pd.read_csv(url, sep=';'))
    except:
        print("Oops! Bei der URL", url, " ist etwas schiefgegangen: ", sys.exc_info()[0])
df_sensor_49366 = pd.concat(list_df_sensor_49366)

In [ ]:
df_sensor_49366.head()

In [ ]:
df_sensor_49366.tail()

Wir legen eine Methode an, um anhand der Daten und der Sensor ID zu checken, ob der Sensor 'indoor' ist. Diese Information wird im Positivfall an die Download URL gehängt. 

In [ ]:
def is_indoor(id: int, data: pd.DataFrame):
    indoor = data[data['sensor.id'] == id]['location.indoor'].unique()[0]
    if indoor == 0:
        return False
    else:
        return True

Obige Schritte fassen wir in zwei Funktionen zusammen, sodass wir sie einfacher verwenden können.

In [ ]:
def get_sensor_data_download_urls(id: int, dates: [str], sensor_type: str, indoor: bool):
    url_front = 'https://archive.sensor.community/'
    indoor_param = ''
    if indoor:
        indoor_param = '_indoor'
    url_back = '_sensor_' + str(id) + indoor_param + '.csv'
    
    # URL Beginn für alle Daten generieren (ohne Sensortyp und Sensor ID)
    list_url = [url_front + date + '/' + date + '_' + sensor_type + url_back for date in dates]
    list_url.pop()
    return list_url

In [ ]:
def get_sensor_data_by_url(urls: [str]):
    list_df_sensor = list()
    for url in urls:
        try:
            result = pd.read_csv(url, sep=';')
            list_df_sensor.append(result)
        except:
            None
    if len(list_df_sensor) == 0:
        return None
    df = pd.concat(list_df_sensor)
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%dT%H:%M:%S')
    return df

### Bielefeld-weite Sensordaten

Über eine API können alle Sensoren in einem GPS-Quadrat abgefragt werden.

In [ ]:
url = 'https://data.sensor.community/airrohr/v1/filter/box=51.9,8.3,52.1,8.7'
res = requests.get(url)
data = json.loads(res.text)
df_sensor_bielefeld = pd.json_normalize(data)
df_sensor_bielefeld.head(3)

Testen wir, ob der zuvor ausgewählte Sensor auch in den Daten enthalten ist.

In [ ]:
df_sensor_bielefeld[df_sensor_bielefeld['sensor.id']==49366]

Wir holen uns alle Sensor IDs für Bielefeld.

In [ ]:
# Alle Sensor IDs für Bielfeld
sensor_ids_bielefeld = df_sensor_bielefeld['sensor.id'].unique()
sensor_ids_bielefeld.sort()

print(sensor_ids_bielefeld)

Da wir nun wissen, wie die Daten aussehen, können wir uns eine Methode schreiben, um den Namen des Sensortypen eines Sensors holen können.

In [ ]:
def get_sensor_type_name_by_id(id: int, data: pd.DataFrame):
    sensor_type = data[data['sensor.id'] == id]['sensor.sensor_type.name']
    sensor_type = sensor_type.iloc[0].lower()
    return sensor_type

Nun können wir uns also die Daten der entsprechenden Sensoren im Bielefelder Stadtgebiet herunterladen und erhalten:

In [ ]:
list_df_sensor_bielefeld = list()
for sensor in sensor_ids_bielefeld[:10]:
    sensor_type = get_sensor_type_name_by_id(int(sensor), df_sensor_bielefeld)
    download_urls = get_sensor_data_download_urls(sensor, list_dates, sensor_type, is_indoor(sensor, df_sensor_bielefeld))
    sensor_data = get_sensor_data_by_url(download_urls)
    list_df_sensor_bielefeld.append(sensor_data)

df_sensor_bielefeld_values = pd.concat(list_df_sensor_bielefeld)
print('Wir haben ' + str(df_sensor_bielefeld_values.shape[0]) + ' Zeilen mit folgendem Inhalt heruntergeladen:')

In [ ]:
df_sensor_bielefeld_values.head()

In [ ]:
df_sensor_bielefeld_values.tail()

Entfernen der nicht validen "nan" Werte...

In [ ]:
df_sensor_bielefeld_values = df_sensor_bielefeld_values.dropna(subset=['sensor_id', 'timestamp', 'P1', 'P2'])
df_sensor_bielefeld_values

Für beide Feinstaubgrößen können wir nun die entsprechenden Zeitreihendiagramme erzeugen:

In [ ]:
# Plot mit P1
fig = px.line(df_sensor_bielefeld_values, x="timestamp", y="P1", color="sensor_id", line_group="sensor_id", hover_name="sensor_id",
        line_shape="linear", render_mode="svg", title='Luftdaten Bielefeld 2021 - PM10')
fig.show()

In [ ]:
# Plot mit P2
fig = px.line(df_sensor_bielefeld_values, x="timestamp", y="P2", color="sensor_id", line_group="sensor_id", hover_name="sensor_id",
        line_shape="linear", render_mode="svg", title='Luftdaten Bielefeld 2021 - PM2.5')
fig.show()

## Analyse der Feinstaubveränderungen in Bielefeld zu Silvester 2019 / 2020

Wir erstellen eine Liste mit den Tagen für Dezember 2019 und Januar 2020.

In [ ]:
from datetime import datetime
start = datetime(2019, 12, 24)
end = datetime(2020, 1, 7)
list_dates_silvester_19_20 = pd.date_range(start, end).tolist()
list_dates_silvester_19_20 = [x.strftime("%Y-%m-%d") for x in list_dates_silvester_19_20]
list_dates_silvester_19_20[:5]

Wir holen uns die Daten für Bielefeld mithilfe dieser Datumsliste

In [ ]:
list_df_sensor_bielefeld_silvester_19_20 = list()
for sensor in sensor_ids_bielefeld[:20]:
    try:
        sensor_type = get_sensor_type_name_by_id(int(sensor), df_sensor_bielefeld)
        download_urls = get_sensor_data_download_urls(sensor, list_dates_silvester_19_20, sensor_type, is_indoor(sensor, df_sensor_bielefeld))
        sensor_data = get_sensor_data_by_url(download_urls)
        if sensor_data is not None:
            list_df_sensor_bielefeld_silvester_19_20.append(sensor_data)
    except:
        None
df_sensor_bielefeld_silvester_19_20 = pd.concat(list_df_sensor_bielefeld_silvester_19_20)
df_sensor_bielefeld_silvester_19_20 = df_sensor_bielefeld_silvester_19_20.dropna(subset=['sensor_id', 'timestamp', 'P1', 'P2'])
print('Wir haben ' + str(df_sensor_bielefeld_silvester_19_20.shape[0]) + ' Zeilen mit folgendem Inhalt heruntergeladen:')


In [ ]:
df_sensor_bielefeld_silvester_19_20.head(3)

In [ ]:
df_sensor_bielefeld_silvester_19_20.tail(3)

Stellen wir also die Feinstaubbelastung im qualitativen Vergleich für Silvester 2019/2020 an:

In [ ]:
# Plot mit P1
fig = px.line(df_sensor_bielefeld_silvester_19_20, x="timestamp", y="P1", color="sensor_id", line_group="sensor_id", hover_name="sensor_id",
        line_shape="linear", render_mode="svg", title='Luftdaten Bielefeld Jahreswechsel 19/20 - PM10')
fig.show()

In [ ]:
# Plot mit P2
fig = px.line(df_sensor_bielefeld_silvester_19_20, x="timestamp", y="P2", color="sensor_id", line_group="sensor_id", hover_name="sensor_id",
        line_shape="linear", render_mode="svg", title='Luftdaten Bielefeld Jahreswechsel 19/20 - PM2.5')
fig.show()

## Zeichnen einer interaktiven Karte

Wir erzeugen zunächst 5-minütliche Durchschnittswerte für alle Sensoren, hier für PM10:

In [ ]:
df_values_1920 = df_sensor_bielefeld_silvester_19_20
df_values_1920['timestamp'] = df_values_1920['timestamp'].dt.round("5min")
df_values_1920_p1 = df_values_1920[['sensor_id', 'P1', 'timestamp']]
df_values_1920_p1 = df_values_1920_p1.groupby(['sensor_id', pd.Grouper(key='timestamp', freq='5Min')])['P1'].mean().to_frame()
df_values_1920_p1 = df_values_1920_p1.reset_index(level=['sensor_id', 'timestamp'])
df_values_1920_p1.head()

Wir fügen nun die GPS-Angaben aus `df_sensor_bielefeld` hinzu:

In [ ]:
df_values_1920_p1 = pd.merge(df_sensor_bielefeld_silvester_19_20, df_sensor_bielefeld[['location.latitude', 'location.longitude', 'sensor.id']], left_on='sensor_id', right_on='sensor.id', how='inner')
df_values_1920_p1 = df_values_1920_p1.drop_duplicates()

Wählen wir nun einen Zeitstempel aus, so können wir ihn zum filtern der verfügbaren Werte nutzen.

In [ ]:
df_values_1920_p1[df_values_1920_p1.timestamp == datetime(2020,1,1,2)]

Noch schnell die hinterlegten GPS-Koordinaten in echte Zahlen konvertiert:

In [ ]:
df_values_1920_p1['location.latitude'] = df_values_1920_p1['location.latitude'].astype(float)
df_values_1920_p1['location.longitude'] = df_values_1920_p1['location.longitude'].astype(float)

Und wir können mit wenig Aufwand eine Karte mit den Standorten der Sensoren erzeugen, inklusive Angabe von PM10-Feinstaub am 1.1.2020 um 2 Uhr:

In [ ]:
fig = px.scatter_mapbox(
    df_values_1920_p1[df_values_1920_p1.timestamp == datetime(2020,1,1,2)], 
    lat="location.latitude", 
    lon="location.longitude", 
    color="P1", 
    size="P1",
    color_continuous_scale=px.colors.cyclical.IceFire, 
    size_max=15, 
    zoom=10,
    mapbox_style="carto-positron"
    )
fig.show()

Mögliche Fortsetzungen gibt es viele: aus einzelnen Bildern ein Video erstellen, analysieren ob/wie die Feinstaubkonzentration am OWD mit Güterzügen oder Berufsverkehr korreliert, ...

Take-away-message:
- Daten müssen teils aufwändig prozessiert und interpretiert werden
- das Potential der Daten liegt beim User


## Linked Data

Bislang haben wir Datenquellen kennen gelernt, welche festgelegte Daten per API oder Download zur Verfügung stellen. Betrachten wir nun *Linked Data*, so erhalten wir ein Vielfaches mehr Möglichkeiten Daten für unseren spezifischen Nutzen zu erhalten - allerdings unter der Voraussetzung, dass wir die jeweilige Quelle und ihre Syntax gut verstehen.

Dies veranschaulichen wir uns am Beispiel von Wikidata.

### Wikidata

Zunächst betrachten wir eines der Beispiele der Wikidata-Seite, nämlich die Entwicklung der Bevölkerungszahlen aller Länder:

In [ ]:
url = 'https://query.wikidata.org/sparql'
query = """
SELECT ?year (AVG(?pop) AS ?population) ?countryLabel
WHERE
{
  ?country wdt:P31 wd:Q6256;
           p:P1082 ?popStatement .
  ?popStatement ps:P1082 ?pop;
                pq:P585 ?date .
  BIND(STR(YEAR(?date)) AS ?year)
  OPTIONAL { ?popStatement pq:P459 ?method. }
  OPTIONAL { ?country p:P1082 [ pq:P585 ?d; pq:P459 ?estimate ].
             FILTER(STR(YEAR(?d)) = ?year). FILTER(?estimate = wd:Q791801). }
  OPTIONAL { ?country p:P1082 [ pq:P585 ?e; pq:P459 ?census ].
             FILTER(STR(YEAR(?e)) = ?year). FILTER(?census = wd:Q39825). }
  OPTIONAL { ?country p:P1082 [ pq:P585 ?f; pq:P459 ?other ].
             FILTER(STR(YEAR(?f)) = ?year). FILTER(?other != wd:Q39825 && ?other != wd:Q791801). }
  BIND(COALESCE( 
    IF(BOUND(?census), ?census, 1/0),
    IF(BOUND(?other), ?other, 1/0),
    IF(BOUND(?estimate), ?estimate, 1/0) ) AS ?pref_method).
  FILTER(IF(BOUND(?pref_method),?method = ?pref_method,true))
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],de" }       
  FILTER(?year >= "2005")
}
GROUP BY ?year ?countryLabel
ORDER BY ?year ?countryLabel
"""
r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()

Als Ergebnis erhalten wir JSON-Schnippsel wie den folgenden:

In [ ]:
data['results']['bindings'][0]

Mit ein paar Code-Zeilen wandeln wir die Daten wiederum in etwas besser Lesbares um:

In [ ]:
from collections import OrderedDict

countries = []
for item in data['results']['bindings']:
    countries.append(OrderedDict({
        'year': item['year']['value'],
        'country': item['countryLabel']['value'],
        'population': item['population']['value']
    }))

df = pd.DataFrame(countries)
df = df.astype({'year': int, 'country': str, 'population': float})
df.head()

Schließlich können wir die Bevölkerungsentwicklung in Deutschland zur Anzeige bringen:

In [ ]:
fig__ger = px.bar(df[df['country'] == 'Deutschland'], x='year', y='population')
fig__ger.show()

Wenn wir uns für lokale Daten interessieren, ist auch dies umstandslos möglich. Über die Suchmaske der Wikidata-Seite können wir das Objekt `Bielefeld` suchen und erhalten die Objekt-ID `Q2112`. Hiermit können wir zum Beispiel alle in Bielefeld geborenen Politiker abrufen:

In [ ]:
query = """
#title: Politicians born in Bielefeld
select distinct ?item ?itemLabel ?itemDescription where {
    ?item wdt:P31 wd:Q5;  # Any instance of a human.
          wdt:P106 wd:Q82955; # Works as politician
          wdt:P19/wdt:P131* wd:Q2112;  #  Who was born in any value (eg. a hospital)
# that has the property of 'administrative area of' Bielefeld or Bielefeld City itself.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "de" }
}
"""
r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()
politicians = []
for item in data['results']['bindings']:
    politicians.append(OrderedDict({
        'Name': item['itemLabel']['value'],
        'Description': item['itemDescription']['value']
    }))
df = pd.DataFrame(politicians)
print('Wir haben ' + str(df.shape[0]) + ' Zeilen mit folgendem Inhalt heruntergeladen:')
df.head()

Aber `SparQL`-Abfragen sind nicht die einzige Möglichkeit an solche Daten zu gelangen. Die Community rundum Wikipedia und Wikidata ist groß, sodass für viele Programmiersprachen Funktionen und Bibliotheken zur Verfügung stehen, welche genutzt werden können. Für Python ist dies zum Beispiel `pywikibot`:

In [ ]:
site = pywikibot.Site("de", "wikipedia")
page = pywikibot.Page(site, "Bielefeld")
item = pywikibot.ItemPage.fromPage(page)

print(item)

Objekte von Wikidata sind wie folgt aufgebaut:

<a href="https://commons.wikimedia.org/wiki/File:Datamodel_in_Wikidata_de.svg#/media/File:Datamodel_in_Wikidata_de.svg"><img src="https://upload.wikimedia.org/wikipedia/commons/1/10/Datamodel_in_Wikidata_de.svg" alt="Datamodel in Wikidata de.svg" width="835" height="600"></a>

Und dieselben Inhalte sind auch an unserem `Bielefeld`-Objekt enthalten:

In [ ]:
item_dict = item.get()
print(item_dict.keys())

Die Aussagen über Bielefeld sind entsprechend in `claims` zu finden:

In [ ]:
clm_dict = item_dict["claims"] # Get the claim dictionary
print(clm_dict)

Und der Inhalt der Aussage für `P190`, welche für Partnerstädte steht, enthält als zweiten Eintrag die folgenden Werte:

In [ ]:
item_dict["claims"]["P190"][1]

Und dies ist die Essenz von *Linked Data*: Objekte sind über Eigenschaften und Relationen miteinander verknüpft. Steigt man über ein Objekt in den Wissensgraphen ein - in unserem Fall war dies *Bielefeld* - so können wir beinahe beliebig tief und verzweigt den Referenzen nachgehen.

Take-away-message:
- das Potenzial von Daten(-quellen) und die Komplexität der Verarbeitung gehen Hand in Hand
- *Linked Data* ist nur so gut wie die Community, welche Daten und Verknüpfungen pflegt
- Communities helfen den Wert der Daten zu erschließen und in die Breite zu tragen

## Ein Nicht-Beispiel

Möchten wir als mündige Bürger und eifrige Online-ShopperInnen die Angaben in dem Impressum einer Webseite überprüfen - wie können wir dies tun? Die Handelsregister führen diese Informationen, aber wie transparent sind die Daten verfügbar?

<a href="https://foundersfoundation.de/impressum/"><img src="img/Impressum_FF.png" alt="Impressum der Founders Foundation" width="835" height="600"></a>

*Hinweis*: Nutze die beiden Seiten Handelsregister und Handelsregisterbekanntmachungen und beachte alle Veröffentlichungen seit Unternehmensgründung.

*Achtung*: Die Informationen liegen in digitaler Form lediglich seit 2006 vor - alle weiter zurückliegenden Daten müssen schriftlich und gegen Gebühr von den Amtsgerichten gekauft werden.

# Danke für eure Aufmerksamkeit!

<a href="https://codefor.de/bielefeld/"><img src="img/CfB_2.png" alt="Code for Bielefeld Promo 2" width="700" align="left"></a>
<a href="https://codefor.de/bielefeld/"><img src="img/CfB_1.png" alt="Code for Bielefeld Promo 1" width="700" align="right"></a>